<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/verbs_cosine_w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!wget rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz


--2020-02-21 13:26:35--  http://rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549952184 (524M) [application/x-gzip]
Saving to: ‘news_mystem_skipgram_1000_20_2015.bin.gz.1’

news_mystem_skipgra 100%[===================>] 524.47M  24.4MB/s    in 23s     

2020-02-21 13:26:58 (22.6 MB/s) - ‘news_mystem_skipgram_1000_20_2015.bin.gz.1’ saved [549952184/549952184]



In [60]:
#!rm collection.json.1 collection.json
!wget https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json

--2020-02-21 13:26:59--  https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 606414 (592K) [text/plain]
Saving to: ‘collection.json.1’

collection.json.1   100%[===================>] 592.20K  --.-KB/s    in 0.06s   

2020-02-21 13:27:00 (9.81 MB/s) - ‘collection.json.1’ saved [606414/606414]



In [0]:
import gensim
from collections import Counter
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances

In [62]:
rv_model = gensim.models.KeyedVectors.load_word2vec_format('news_mystem_skipgram_1000_20_2015.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
collection_handler = open("collection.json", "r", encoding="utf-8")
collection = json.load(collection_handler, encoding="utf-8")

In [67]:

def make_group(polarity_type_list):
  """
  На вход подаются пары из разных типов отношений(p_type) в списках(как в переменной pol_type_dict)
  для каждой пары цикл ищет не более ста глаголов, у которых есть типы отношений, совпадающие с нашей парой, 
  которая подается на вход. 
  """
  group_dict = Counter()
  for p_type in polarity_type_list:
    print(p_type)
    count = 0
    for key in collection.keys():
      if count < 100:
        try:
          #print(collection[key]['frames']['polarity'])
          if p_type[0] in collection[key]['frames']['polarity'] and p_type[1] in collection[key]['frames']['polarity'] :
          # эту строчку можно менять, если нам нужно искать отношения, где только ['a0', 'a1', 'pos', 1.0]:
          # if p_type[0] in collection[key]['frames']['polarity'] and p_type[1] not in collection[key]['frames']['polarity'] : 
          # проблема! в лексиконе всего 20 отношений [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]] c разными коэффициентами, и 51 таких, 
          # где ['a0', 'a1', 'pos', 1.0], а ['a1', 'a0', 'pos', 1.0] отсутствует
            for ver in collection[key]['title']:
              group_dict[ver] = p_type
              count += 1
            for var in collection[key]['variants']:
              if len(var.split(' ')) > 1 and var[-2:] == 'ть':
                group_dict[var] = p_type
                count += 1

              
        except Exception as e:
          pass
  return group_dict

pol_type_dict = [[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]], \
                 [['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]], \
                  [['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]], \
                  [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]],
                 ]

groups_total_dict = make_group(pol_type_dict)
len(groups_total_dict)
# make_group(pol_type_dict)


[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]
[['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]]
[['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]]
[['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]]


169

In [68]:
groups_total_dict.most_common(2)

[('помочь', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]),
 ('поощрить', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]])]

In [0]:
verbs_dict = ['одобрять_V', 'хвалить_V', 'поощрять_V']
def make_sid_embedding(seed_dict, model):
  '''
  Возвращает вектор сидового множества(не двумерный)
  '''
  summ = 0
  try:
    for verb in seed_dict:
      #print(model[verb])
      summ = summ + model[verb]
  except KeyError:
    pass
  vector = summ/len(seed_dict)
  return vector
opinion_embedding = make_sid_embedding(opinion_dict, rv_model)


In [0]:
def ma
verbs = [verb for verb in groups_total_dict.keys()]
polaryties = [pol for pol in groups_total_dict.values()]
df_positive = pd.DataFrame({'verbs':verbs, 'polarity':polaryties})

In [72]:
df_positive.head(7)

,verbs,polarity
0,помочь,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]"
1,поощрить,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]"
2,похвалить,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]"
3,приветствовать,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]"
4,радовать,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]"
5,дарить радость,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]"
6,доставить радость,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]"


In [0]:
def add_sim_list(model, res_column, verbs_dict, sid_vector, verb_mark = '_V')
import pytmorphy
  '''
  Принимает на вход модель, как назвать результирующую колонку в датафрейме,
  словарь из глаголов, средний вектор сидового множества, и обозначение глагола в модели(если оно есть).
  записывает колонку в датафрейм, возвращает словарь косинусных расстояний
  '''
  sim_list = []  
  for verb in verbs:
    try:
      similarity = cosine_distances(np.atleast_2d(rv_model[verb + verb_mark]), np.atleast_2d(opinion_embedding))
    except KeyError:
      similarity = 'verb is not in model dict'
    sim_list.append(similarity)
  df_positive[res_column] = sim_list
  return sim_list

sim_list(rv_model, res_column, verbs_dict, sid_vector)

In [82]:
df_positive.head(10)

,verbs,polarity,distance_with_sid
0,помочь,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",verb is not in model dict
1,поощрить,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",verb is not in model dict
2,похвалить,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",[[0.48269153]]
3,приветствовать,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",[[0.5898497]]
4,радовать,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",[[0.72734857]]
5,дарить радость,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",verb is not in model dict
6,доставить радость,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",verb is not in model dict
7,доставлять радость,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",verb is not in model dict
8,подарить радость,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",verb is not in model dict
9,спасти,"[[a0, a1, pos, 1.0], [a1, a0, pos, 1.0]]",verb is not in model dict


In [58]:
# vect1 = np.atleast_2d([1, 2, 4])
# vect2 = np.atleast_2d([1, 4, 4])
# print(cosine_distances(vect1, vect2))
# vect1

[[0.05032853]]


array([[1, 2, 4]])